## Токенизация

Задача - разделить предложение на слова или отдельные элементы (знаки препинания, гиперссылки и т.д.), по возможности сохраняя какие-то атрибуты текста.

# Регурярные выражения

В модуле `re` есть недокументированный класс `Scanner`, с помощью которого можно реализовать лексический анализатор. `Scanner` будет искать вхождения паттернов в тексте и на каждое совпадение вызывать соответствующую функцию. В общем случае подобный код неэффективен, лексические анализаторы лучше реализовывать с помощью специальных инструментов - генераторов лексических анализаторов, которые обеспечат анализ за линейное время.

In [2]:
import re

scanner = re.Scanner(
   [(r'(\w+)@(\w+)\.(\w{2,3})', lambda s, x: (x, 'email')),
    (r'[a-zA-Z]+', lambda s, x: (x, 'word')), 
    (r'\d+', lambda s, x: (x, 'digit')),    
    (r'\s+', lambda s, x: (x, 'whitespace')),
    (r'[.,;"!?:]', lambda s, x: (x, 'preposition')),
    ])

##scanner.scan('hello, world 1234 test@example.com')
scanner.scan("Hello world!")

([('Hello', 'word'),
  (' ', 'whitespace'),
  ('world', 'word'),
  ('!', 'preposition')],
 '')

## NLTK
Natural Language Toolkit, библиотека для обработки естественных языков. Она создавалась для учебных целей, но тем не менее приобрела определенную популярность. Реализовано некоторое количество методов токенизации, которые можно использовать для повседневных задач и экспериментов.

In [6]:
import nltk
from nltk.tokenize import wordpunct_tokenize, word_tokenize, TweetTokenizer

nltk.download('punkt')

tweet_tokenize = TweetTokenizer()

sentences = ["Hello world 4.2.", "LA New-York", "Hello world 4.2!", "Say me #hello"]

for sent in sentences:
    print("Sentence: {}".format(sent))
    print("word_tokenize: ", word_tokenize(sent))
    print("wordpunct_tokenize: ", wordpunct_tokenize(sent)),
    print("tweet: ", tweet_tokenize.tokenize(sent))
    print()

Sentence: Hello world 4.2.
word_tokenize:  ['Hello', 'world', '4.2', '.']
wordpunct_tokenize:  ['Hello', 'world', '4', '.', '2', '.']
tweet:  ['Hello', 'world', '4.2', '.']

Sentence: LA New-York
word_tokenize:  ['LA', 'New-York']
wordpunct_tokenize:  ['LA', 'New', '-', 'York']
tweet:  ['LA', 'New-York']

Sentence: Hello world 4.2!
word_tokenize:  ['Hello', 'world', '4.2', '!']
wordpunct_tokenize:  ['Hello', 'world', '4', '.', '2', '!']
tweet:  ['Hello', 'world', '4.2', '!']

Sentence: Say me #hello
word_tokenize:  ['Say', 'me', '#', 'hello']
wordpunct_tokenize:  ['Say', 'me', '#', 'hello']
tweet:  ['Say', 'me', '#hello']



[nltk_data] Downloading package punkt to /home/alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Ply

Приведем лексического анализатора на `ply`. В данном случае анализатор описывается в классе, могут быть три вида токенов - слова, цифры и пробелы. Для каждого токена в тексте выозвращается необходимая информация - типа, длина смещение:

In [7]:
from ply.lex import lex, TOKEN

class Lexer:
    tokens = ( 'NUMBER', 'ID', 'WHITESPACE' )
    
    @TOKEN(r'\d{1,5}')
    def t_NUMBER(self, t):
        t.value = int(t.value)
        return t

    @TOKEN(r'\w+')
    def t_ID(self, t):
        return t

    @TOKEN(r'\s+')
    def t_WHITESPACE(self, t):
        pass

    def t_error(self, t):
        pass
    

__file__ = "02.Tokenizers.ipynb"    # make `ply` happy

lexer = lex(object=Lexer())
lexer.input('123 abs 965')
for token in lexer:
    print(token)

LexToken(NUMBER,123,1,0)
LexToken(ID,'abs',1,4)
LexToken(NUMBER,965,1,8)


### Pyparsing

Другой пример pyparsing, с помощью которого можно обрабатывать более широкий класс формальных языков. С помощью специального DSL (domain-specific language, предметно-ориентированный язык) описывается грамматика. С помощью pyparsing можно обрабатывать коллекции в специфичных форматах, извлекать логи и так далее.

Опишем грамматику, котора позволяет разобрать записи вида: 
>   <слово>: число, число, ...

In [8]:
from pyparsing import Word, alphas, nums,  Literal, StringEnd, ZeroOrMore, Suppress, OneOrMore 

word = Word(alphas)
num = Word(nums)
sep = Suppress(OneOrMore(','))
col = Suppress(':')

s = word + col + num + ZeroOrMore(sep + num) + StringEnd()
        
s.parseString('hello: 1, 22, 3')

(['hello', '1', '22', '3'], {})

Здесь более сложный пример, грамматика описывает правильные скобочные записи.


In [9]:
from pyparsing import Literal, Forward, StringEnd, OneOrMore, Empty

br_o = Literal('(')
br_c = Literal(')')

braces = Forward()
braces << OneOrMore(br_o + (braces | Empty() ) + br_c)
start = braces + StringEnd()
        
start.parseString('(())()()')

(['(', '(', ')', ')', '(', ')', '(', ')'], {})

Опишем грамматику для разбора простейших математических выражений. Сначала классы, которые описывают узлы деревьев разбора:

In [10]:
from pyparsing import Word, Literal, Or, nums, Forward, StringEnd
from operator import mul, truediv, add, sub

class NumNode(object):
    def __init__(self, t):
        self.num = float(t[0])        
    def calc(self):
        return self.num          
    def __repr__(self):
        return 'Num(%s)' % self.num
        
class OpNode(object):
    def __init__(self, t):               
        self.left = t[0]
        self.op = { '-' : sub, '+' : add, '/' : truediv, '*' : mul }[t[1]]
        self.right = t[2]       
    def calc(self):
        return self.op(self.left.calc(), self.right.calc())        
    def __repr__(self):
        return 'Op(%s, %s, %s)' % (self.left, self.op, self.right)

затем опишем грамматику

In [11]:
plus = Literal('+')
minus = Literal('-')
div = Literal('/')
mult = Literal('*')
        
factor = Word(nums).setParseAction(NumNode)

term = Forward()
term << (( factor + (mult | div) + term ).setParseAction(OpNode) | factor )        

expr = Forward()
expr << ((term + (plus | minus) + expr).setParseAction(OpNode) | term )

start = expr + StringEnd()

In [12]:
tree = start.parseString('2 * 4 + 6 * 7')[0]
print(tree)
print(tree.calc())

Op(Op(Num(2.0), <built-in function mul>, Num(4.0)), <built-in function add>, Op(Num(6.0), <built-in function mul>, Num(7.0)))
50.0
